<a href="https://colab.research.google.com/github/EAName/yelp-reviews/blob/main/Create_Combined_Business_and_Review_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import statistics

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 


In [14]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

## Mount Google Drive to Colab environment

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load Business Dataset

In [3]:
df_stlouis =pd.read_csv('/content/gdrive/MyDrive/yelp_dataset/business_df_stlouis.csv')

In [4]:
df_stlouis.shape

(3181, 69)

In [5]:
df_stlouis.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Chicken Wings,Sports Bars,JapaneseTotal,HasTV_NA,RestaurantsPriceRange2_NA,RestaurantsReservations_NA,garage_NA,validated_NA,lot_NA,valet_NA
0,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123.0,38.565165,-90.321087,3.0,19,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Alton,IL,62002.0,38.896563,-90.186203,3.0,94,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1MeIwdbTnZOBFCKOrgaxuw,Ricardo's Italian Cafe,1931 Park Ave,Saint Louis,MO,63104.0,38.617272,-90.212784,3.5,80,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,VrGxoSNZfSiT-G4OqRRGIw,Sunset 44 Bistro,118 W Adams Ave,Saint Louis,MO,63122.0,38.583223,-90.407187,4.0,73,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,wjyUxHxkrvUvCOVqy4g_wg,Oscar's Cafe,7520 Natural Bridge Rd,Normandy,MO,63121.0,38.704765,-90.299811,4.0,11,...,0,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


## Load Reviews Dataset

In [8]:
df =pd.read_csv('/content/gdrive/MyDrive/yelp_dataset/yelp_reviews_clean_version4_st_louis.csv')

In [9]:
df = df.groupby(['business_id'])['text'].apply(lambda x: ','.join(x)).reset_index()

In [10]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [11]:
df.head(2)

business_id  \
0  -1wnOZFsvsJPEXh_hWcmMA   
1  -2gmbMDzKgYZ_8DOnJtPyw   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [12]:
def clean_doc(doc): 
    #split review into individual words
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]

    #word lemmzatization    
    lemm = WordNetLemmatizer()
    tokens=[lemm.lemmatize(word) for word in tokens]

    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 3]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]     

    # filter out common words
    tokens = [w for w in tokens if not w in ['louis', 'restaurant', 'also', 'youre']] 
    # # word stemming    
    # ps=PorterStemmer()
    # tokens=[ps.stem(word) for word in tokens]

    return tokens

def final_processed_text(doc):
    #this is a function to join the processed text back
    ' '.join(doc)
    return doc

In [15]:
df['cleaned_text'] = df['text'].apply(lambda x: clean_doc(x))

final_processed_text = [' '.join(x) for x in df['cleaned_text'].tolist()]
df['final_processed_text'] = final_processed_text     #add to dataframe

In [16]:
df.head(2)

business_id  \
0  -1wnOZFsvsJPEXh_hWcmMA   
1  -2gmbMDzKgYZ_8DOnJtPyw   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [17]:
combined = pd.merge(df, df_stlouis, on ='business_id')

In [18]:
combined.head(2)

business_id  \
0  -1wnOZFsvsJPEXh_hWcmMA   
1  -2gmbMDzKgYZ_8DOnJtPyw   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [19]:
combined.shape

(3181, 72)

In [20]:
combined.to_csv('/content/gdrive/MyDrive/yelp_dataset/business_review_combined_version1.csv', index = False)